# GSE GPWR Simulator Demo

Interactive demonstration of the GSE GPWR Python library.

**Requirements:**
- Simulator running on Windows VM (10.1.0.123)
- GDA Server listening on port 9800

**To start simulator:**
```bash
ssh brad@10.1.0.123
cd D:\GPWR\Plant
call UploadGPWR_EnglishUnit_ALL.cmd
```

## 1. Import Libraries

In [ ]:
from gse import GDAClient, GPWREnvironment
import time
import socket

print("✓ GSE library imported successfully")

## 2. Test Network Connection

In [ ]:
# Quick connectivity test
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(5.0)
    sock.connect(('10.1.0.123', 9800))
    sock.close()
    print("✓ Simulator is reachable at 10.1.0.123:9800")
except Exception as e:
    print(f"✗ Cannot reach simulator: {e}")
    print("\nMake sure simulator is running on Windows VM")

## 3. Connect to GDA Server

In [ ]:
# Create client and connect
client = GDAClient(host='10.1.0.123', port=9800, timeout=10.0)
client.connect()

print("✓ Connected to GDA Server")

## 4. Read Single Variables

In [ ]:
# Read reactor power
power = client.read_variable('RCS01POWER')
print(f"Reactor Power: {power} MW")

# Read average temperature
tavg = client.read_variable('RCS01TAVE')
print(f"Average Temperature: {tavg} °F")

# Read pressurizer pressure
press = client.read_variable('PRS01PRESS')
print(f"Pressurizer Pressure: {press} psia")

# Read steam generator level
sg_level = client.read_variable('SGN01LEVEL')
print(f"SG 1 Level: {sg_level} %")

## 5. Batch Read Multiple Variables

In [ ]:
# Read multiple variables at once (more efficient)
variables = [
    'RCS01POWER',    # Reactor power
    'RCS01TAVE',     # Average temp
    'RCS01THOT',     # Hot leg temp
    'RCS01TCOLD',    # Cold leg temp
    'PRS01PRESS',    # Pressurizer pressure
    'PRS01LEVEL',    # Pressurizer level
    'SGN01LEVEL',    # SG 1 level
    'SGN02LEVEL',    # SG 2 level
    'TUR01SPEED',    # Turbine speed
    'GEN01POWER',    # Generator power
]

values = client.read_variables(variables)

print("\nCurrent Plant State:")
print("=" * 50)
for var, val in zip(variables, values):
    print(f"{var:20s} = {val}")

## 6. Write Variables (Control Actions)

In [ ]:
# Read current control rod position
current_rods = client.read_variable('RTC01DEMAND')
print(f"Current rod position: {current_rods}")

# Write the same value back (safe test - no change)
client.write_variable('RTC01DEMAND', float(current_rods))
print(f"✓ Successfully wrote rod position: {current_rods}")

# Verify
new_rods = client.read_variable('RTC01DEMAND')
print(f"Verified rod position: {new_rods}")

## 7. Monitor Variables Over Time

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Collect data for 30 seconds
times = []
powers = []
temps = []

print("Collecting data for 30 seconds...")
start_time = time.time()

while time.time() - start_time < 30:
    t = time.time() - start_time
    power = float(client.read_variable('RCS01POWER'))
    temp = float(client.read_variable('RCS01TAVE'))
    
    times.append(t)
    powers.append(power)
    temps.append(temp)
    
    time.sleep(1.0)  # Sample every second

# Plot results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

ax1.plot(times, powers, 'b-', linewidth=2)
ax1.set_ylabel('Reactor Power (MW)', fontsize=12)
ax1.set_title('GSE GPWR Simulator - Real-Time Data', fontsize=14)
ax1.grid(True, alpha=0.3)

ax2.plot(times, temps, 'r-', linewidth=2)
ax2.set_xlabel('Time (seconds)', fontsize=12)
ax2.set_ylabel('Average Temperature (°F)', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ Collected {len(times)} data points")

## 8. Reset to Initial Condition

In [ ]:
# Save current power
before_power = client.read_variable('RCS01POWER')
print(f"Power before reset: {before_power} MW")

# Reset to 100% power IC
print("\nResetting to 100% power initial condition...")
print("(This takes 2-5 seconds)")

client.reset_to_ic(100)

# Wait for reset to complete
time.sleep(3)

# Read new power
after_power = client.read_variable('RCS01POWER')
print(f"\nPower after reset: {after_power} MW")
print("✓ Reset complete")

## 9. Insert Malfunction (Equipment Failure)

In [ ]:
# WARNING: This will actually affect the simulation!
# Uncomment to test malfunction insertion

# # Read current pump speed
# pump_speed = client.read_variable('RCS01PUMP1SPD')
# print(f"Current pump 1 speed: {pump_speed}")

# # Insert gradual pump failure
# print("\nInserting pump failure (reducing to 80% over 10 seconds)...")
# malf_idx = client.insert_malfunction(
#     var_name='RCS01PUMP1SPD',
#     final_value=80.0,
#     ramp_time=10,
#     delay=0
# )
# print(f"✓ Malfunction inserted (index: {malf_idx})")

# # Monitor pump speed during failure
# for i in range(12):
#     speed = client.read_variable('RCS01PUMP1SPD')
#     print(f"  t={i}s: Pump speed = {speed}")
#     time.sleep(1)

print("Malfunction test commented out - uncomment to try it")

## 10. RL Environment Interface

In [ ]:
# Disconnect low-level client
client.disconnect()

# Create RL environment
env = GPWREnvironment(host='10.1.0.123', port=9800)

# Connect
env.__enter__()

print("✓ RL Environment created")
print(f"  Observation space: {len(env.obs_vars)} variables")
print(f"  Action space: {len(env.action_vars)} variables")

In [ ]:
# Reset environment
obs = env.reset(ic=100)

print("\nInitial Observations:")
print("=" * 50)
for key, value in obs.items():
    print(f"{key:20s} = {value:.2f}")

In [ ]:
# Run a simple episode
print("Running 10-step episode...\n")

total_reward = 0

for step in range(10):
    # Simple policy: no control actions (maintain current state)
    action = {
        'rod_demand': 0.0,
        'przr_spray': 0.0,
        'przr_heaters': 50.0,
        'fw_flow_demand': 100.0,
        'turbine_governor': 100.0,
    }
    
    # Take step
    obs, reward, done, info = env.step(action)
    total_reward += reward
    
    print(f"Step {step+1}: Power={obs['reactor_power']:.1f} MW, Reward={reward:.2f}")
    
    if done:
        print("Episode terminated (trip or limit exceeded)")
        break

print(f"\nTotal Reward: {total_reward:.2f}")

In [ ]:
# Cleanup
env.__exit__(None, None, None)
print("✓ Environment closed")

## Summary

You've successfully:
- ✓ Connected to GSE GPWR simulator from Linux
- ✓ Read individual and batch variables
- ✓ Written control commands
- ✓ Monitored real-time data
- ✓ Reset to initial conditions
- ✓ Used the RL environment interface

**You're ready to train RL agents!**

Next steps:
1. Customize observation/action spaces in `GPWREnvironment`
2. Define your reward function
3. Integrate with your RL framework (Stable-Baselines3, Ray RLlib, etc.)
4. Start training!